In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import shapely

cellrebel_gpd =pd.read_hdf('consolidate_db.h5')

In [ ]:
xmin, ymin, xmax, ymax = cellrebel_gpd.total_bounds

length =  0.0008985 * 10  ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)
wide =  0.0008985 * 10 ### (the distance for 100 metres is 0.0008985, so if you want to get 250 metres, need to multiply 2.5)

cols = list(np.arange(xmin, xmax + wide, wide))
rows = list(np.arange(ymin, ymax + length, length))

polygons = []
for x in cols[:-1]: ### [:-1] these are used stop -1 of position from the end of list 
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=4326)

grid.reset_index(inplace=True)
grid.rename(columns={'index':'id'}, inplace = True)

In [ ]:
cellrebel_gpd = gpd.sjoin(left_df= cellrebel_gpd, right_df=grid, how='left', predicate='within')

In [ ]:
cellrebel_gpd = pd.pivot_table(cellrebel_gpd, values = 'reference_signal_received_power', index = 'geometry1', columns = 'mobile_operator', aggfunc= len)

In [ ]:
cellrebel_gpd.reset_index(inplace=True, col_level=1, allow_duplicates=False)

In [ ]:
cellrebel_gpd.replace(np.nan, 0, inplace=True)

In [ ]:
cellrebel_gpd['Grand Total'] = cellrebel_gpd['3'] + cellrebel_gpd['Indosat Ooredoo'] + cellrebel_gpd['PT Telekomunikasi Indonesia'] + cellrebel_gpd['Smartfren'] + cellrebel_gpd['Telkomsel'] + cellrebel_gpd['XL Axiata'] 

In [ ]:
cellrebel_gpd.to_csv('result_20230505.csv', index=False)

In [ ]:
cellrebel_gpd = gpd.GeoDataFrame(cellrebel_gpd, geometry=gpd.GeoSeries.from_wkt(cellrebel_gpd['geometry1']), crs=4326)

In [ ]:
cellrebel_gpd.to_file('cellrebel_population.tab', driver='MapInfo File')